---
---
---

# MAP CADDv1.4 annotations (for detected & non-detected CK pos) to dbNSFP score files

1.02.20 mfpfox on local, other M part on hoffman- origianl version with MISSENSE CADD file filtering from 12/27/19

### VEP VERSION NOTE: 
    - CADDv1.4 using VEPv92
    - dbNSFP using VEPv94

**hoffman PATH : /u/home/m/mfpalafo/project-arboleda/CADD/PULL_annotations**
    * DIR downloaded locally for part 3 QC "RESULT_pos_overlap_dbNSFPcoordinates/"
    * hoffman files have all consequences

**hoffman M markdown has code for:** 
- [1] chr chunks
- [2] search chromosome position against CADD files

**this M markdwon has code for:** 
- [3] CADDv1.4 files format AA col and pos_id col with leading 0's
- [4] simplify dbNSFP cols and parse list columm values based on 'matched_index' column
- [5] merge cadd chr files on pos_id column from dbNSFP
    - detected = 104,475 
    - not detect = 1,222,911
- [6] merge CADD chr files
- [7] CADD+dbNSFP annotation files


# [3] adding {pos_id19 | pos_id38} {Amino_acids} col to CADD chr files filtered for hg19 | hg38 position

PATH = '/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CADDmapped/RESULT_CADDv14_pos_overlap_dbNSFPcoordinates/ALL_CONSEQUENCES/'

- initially used MISSENSE filtered file and recovered 20k less detected positions.
- low yeild do to CADD using VEP v92 while dbNSFP changes and annotations from v94 VEP
- redoing CADD file parsing using ALL consequences (not filtering on Consequence column which happens in later code of this markdown)

```python

# !/usr/bin/env python3
# -*- coding: utf-8 -*-

# Pmap M local ipynb code, Pmap_missense_annotations_QC.py
# markdown M local QC of positions from dbNSFP overlapped with CADD37 or 38 annotations

import os
import sys
import pandas as pd


def create_coordinate_id(df, chrr, pos, ref, alt, assembly):
    if assembly == 37:
        df.loc[:,'pos_id19'] = df[chrr].astype(str) + '_' + \
                df[pos].astype(str) + '_' + df[ref].astype(str) + \
                '_' + df[alt].astype(str)
    if assembly == 38:
        df.loc[:,'pos_id38'] = df[chrr].astype(str) + '_' + \
                df[pos].astype(str) + '_' + df[ref].astype(str) + \
                '_' + df[alt].astype(str)
    return df


def format_missense_triple(df, oaacol, naacol):
    #  A|A turns to Ala/Ala
    amino_dict = dict([('A', 'Ala'),('G', 'Gly'), ('I','Ile'), ('L','Leu'), ('P', 'Pro'), ('V','Val'), ('F','Phe'),('W', 'Trp'), ('Y', 'Tyr'), ('D','Asp'),('E','Glu'), ('R','Arg'),('H','His'), ('K','Lys'), ('S','Ser'), ('T', 'Thr'), ('C', 'Cys'), ('M', 'Met'), ('N', 'Asn'), ('Q','Gln')])
    df[oaacol].replace(amino_dict, inplace=True)
    df[naacol].replace(amino_dict, inplace=True)
    ccopy = df[naacol].copy()
    df['Amino_acids'] = df[oaacol].str.cat(ccopy, sep='/')
    return df


def filter_cadd_overlap(df, assembly):
    # deleted [1] section that did missense filtering step
    # [2] new pos_id(assembly) to files
    if assembly == 37:
        df = create_coordinate_id(df, 'chr', 'pos_hg19', 'Ref', 'Alt', assembly)
    if assembly == 38:
        df = create_coordinate_id(df, 'chr', 'pos_hg38', 'Ref', 'Alt', assembly)
    # [3] new missense type column in 3 letter format with '/' sep {oAA, nAA}
    df = format_missense_triple(df, 'oAA', 'nAA')
    return df


def main():
    os.chdir('/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CADDmapped/RESULT_CADDv14_pos_overlap_dbNSFPcoordinates/ALL_CONSEQUENCES/')

    chrls = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X','Y']

    # GRCh37
    for order in chrls:
        chrID = 'chr{}'.format(order)
        file1 = '{}_CADD_GRCh37_DETECTED_CK.csv'.format(chrID)
        out1 = 'ALLCON_{}_CADD_GRCh37_DETECTED_CK.csv'.format(chrID)
        file2 = '{}_CADD_GRCh37_NOT_DETECTED_CK.csv'.format(chrID)
        out2 = 'ALLCON_{}_CADD_GRCh37_NOT_DETECTED_CK.csv'.format(chrID)
        df1 = pd.read_csv(file1, low_memory=False, converters={'pos_hg19': '{:0>9}'.format})
        df2 = pd.read_csv(file2, low_memory=False, converters={'pos_hg19': '{:0>9}'.format})
        df1out = filter_cadd_overlap(df1, 37)
        df2out = filter_cadd_overlap(df2, 37)
        print("saving detected and not detected GRCh37 ", chrID)
        print()
        df1out.to_csv(out1, index=False)
        df2out.to_csv(out2, index=False)

    # GRCh38
    for order in chrls:
        chrID = 'chr{}'.format(order)
        file1 = '{}_CADD_GRCh38_DETECTED_CK.csv'.format(chrID)
        out1 = 'ALLCON_{}_CADD_GRCh38_DETECTED_CK.csv'.format(chrID)
        file2 = '{}_CADD_GRCh38_NOT_DETECTED_CK.csv'.format(chrID)
        out2 = 'ALLCON_{}_CADD_GRCh38_NOT_DETECTED_CK.csv'.format(chrID)
        df1 = pd.read_csv(file1, low_memory=False, converters={'pos_hg38': '{:0>9}'.format})
        df2 = pd.read_csv(file2, low_memory=False, converters={'pos_hg38': '{:0>9}'.format})
        df1out = filter_cadd_overlap(df1, 38)
        df2out = filter_cadd_overlap(df2, 38)
        print("saving detected and not detected GRCh38 ", chrID)
        print()
        df1out.to_csv(out1, index=False)
        df2out.to_csv(out2, index=False)

main()
```

# dbNSFP pos_id extracted as single column and used next to filter CADD ALL ANNOTATION files on inner merge 

path = /Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CADDmapped/BUG_FIX
```bash
#!/bin/bash
# keeps header and appends all rows together
# for Pmap dbNSFP CADD mapped detected and non detected chr merge

cut -d',' -f1 < SCORE_dbNSFP_selectcols_NOT_detected_CK_1222911.csv > col_notdet_posid19.csv

cut -d',' -f2 < SCORE_dbNSFP_selectcols_NOT_detected_CK_1222911.csv > col_notdet_posid38.csv

cut -d',' -f1 < SCORE_dbNSFP_selectcols_detected_CK_104475.csv > col_det_posid19.csv

cut -d',' -f2 < SCORE_dbNSFP_selectcols_detected_CK_104475.csv > col_det_posid38.csv

```

---
---
---



# [4] simplifying dbNSFP score files by COLUMNS OF INTEREST

rules: 
1. merge on pos_idcoordinates
2. match on Amino_acids 'A/A'
3. protein positions match- not required since most annotations refer to coordinates for annotation


```python
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CADDmapped/BUG_FIX/")

##  detected dbNSFP score file: 
det_scores = 'SCORE_annotation_3840_CYS_LYS_detected.csv'
detscore = pd.read_csv(det_scores, low_memory=False)
print(detscore.shape)
# (104475, 400)

### simplify df with select columns: 
det_cols = ['pos_id19', 'pos_id38_x', 'aaref_x', 'aaalt_x', 'matched_aapos_x',
       'CADD_phred_hg38', 'matched_UKBID_x', 'pos_dict', 'matched_target',
       'CADD_phred_hg19', 'CADDdiff_38minus19', 'Amino_acids', 'pos_ID',
       'Cys_reactivity', 'Cys_react_threshold', 'Cys_target_label',
       'Lys_reactivity', 'Lys_react_threshold', 'Lys_target_label', 'rs_dbSNP151', 'Ensembl_transcriptid','cds_strand', 'refcodon', 'codonpos', 'codon_degeneracy',
       'SIFT_score', 'SIFT_converted_rankscore', 'SIFT_pred', 'SIFT4G_score',
       'SIFT4G_converted_rankscore', 'SIFT4G_pred', 'Polyphen2_HDIV_score',
       'Polyphen2_HDIV_rankscore', 'Polyphen2_HDIV_pred',
       'Polyphen2_HVAR_score', 'Polyphen2_HVAR_rankscore',
       'Polyphen2_HVAR_pred','LRT_score', 'LRT_converted_rankscore','LRT_pred', 'LRT_Omega', 'MutationTaster_score','MutationTaster_converted_rankscore', 'MutationTaster_pred',
       'MutationTaster_model', 'MutationTaster_AAE', 'MutationAssessor_score',
       'MutationAssessor_rankscore', 'MutationAssessor_pred',
       'FATHMM_score','FATHMM_converted_rankscore', 'FATHMM_pred', 'PROVEAN_score','PROVEAN_converted_rankscore', 'PROVEAN_pred', 'VEST4_score','VEST4_rankscore', 'MetaSVM_score', 'MetaSVM_rankscore', 'MetaSVM_pred','MetaLR_score', 'MetaLR_rankscore', 'MetaLR_pred', 'Reliability_index','M-CAP_score', 'M-CAP_rankscore', 'M-CAP_pred', 'REVEL_score','REVEL_rankscore', 'MutPred_score', 'MutPred_rankscore',
       'MutPred_protID', 'MutPred_AAchange', 'MutPred_Top5features',
        'MPC_score','MPC_rankscore','PrimateAI_score', 'PrimateAI_rankscore', 'PrimateAI_pred','DANN_score', 'DANN_rankscore', 'fathmm-MKL_coding_score','fathmm-MKL_coding_rankscore', 'fathmm-MKL_coding_pred','fathmm-MKL_coding_group', 'fathmm-XF_coding_score',
       'fathmm-XF_coding_rankscore', 'fathmm-XF_coding_pred',
       'Eigen-raw_coding', 'Eigen-raw_coding_rankscore', 'Eigen-pred_coding',
       'Eigen-PC-raw_coding', 'Eigen-PC-raw_coding_rankscore',
       'Eigen-PC-phred_coding', 'matched_index']
# filter
simpleD = detscore[det_cols].copy()

rename = ['pos_id19', 'pos_id38', 'aaref', 'aaalt', 'matched_aapos',
       'CADD_phred_hg38', 'matched_UKBID', 'pos_dict', 'matched_target',
       'CADD_phred_hg19', 'CADDdiff_38minus19', 'Amino_acids', 'pos_ID',
       'Cys_reactivity', 'Cys_react_threshold', 'Cys_target_label',
       'Lys_reactivity', 'Lys_react_threshold', 'Lys_target_label', 'rs_dbSNP151', 'Ensembl_transcriptid','cds_strand', 'refcodon', 'codonpos', 'codon_degeneracy','SIFT_score', 'SIFT_converted_rankscore', 'SIFT_pred', 'SIFT4G_score',
       'SIFT4G_converted_rankscore', 'SIFT4G_pred', 'Polyphen2_HDIV_score',
       'Polyphen2_HDIV_rankscore', 'Polyphen2_HDIV_pred',
       'Polyphen2_HVAR_score', 'Polyphen2_HVAR_rankscore',
       'Polyphen2_HVAR_pred','LRT_score', 'LRT_converted_rankscore','LRT_pred', 'LRT_Omega', 'MutationTaster_score','MutationTaster_converted_rankscore', 'MutationTaster_pred','MutationTaster_model', 'MutationTaster_AAE', 'MutationAssessor_score','MutationAssessor_rankscore', 'MutationAssessor_pred',
       'FATHMM_score','FATHMM_converted_rankscore', 'FATHMM_pred', 'PROVEAN_score','PROVEAN_converted_rankscore', 'PROVEAN_pred', 'VEST4_score','VEST4_rankscore', 'MetaSVM_score', 'MetaSVM_rankscore', 'MetaSVM_pred','MetaLR_score', 'MetaLR_rankscore', 'MetaLR_pred', 'Reliability_index','M-CAP_score', 'M-CAP_rankscore', 'M-CAP_pred', 'REVEL_score','REVEL_rankscore', 'MutPred_score', 'MutPred_rankscore',
       'MutPred_protID', 'MutPred_AAchange', 'MutPred_Top5features',
        'MPC_score','MPC_rankscore','PrimateAI_score', 'PrimateAI_rankscore', 'PrimateAI_pred','DANN_score', 'DANN_rankscore', 'fathmm-MKL_coding_score','fathmm-MKL_coding_rankscore', 'fathmm-MKL_coding_pred','fathmm-MKL_coding_group', 'fathmm-XF_coding_score','fathmm-XF_coding_rankscore', 'fathmm-XF_coding_pred','Eigen-raw_coding', 'Eigen-raw_coding_rankscore', 'Eigen-pred_coding',
       'Eigen-PC-raw_coding', 'Eigen-PC-raw_coding_rankscore',
       'Eigen-PC-phred_coding', 'matched_index']

simpleD.columns = rename

# change order
neworder = ['pos_id19', 'pos_id38', 'pos_ID', 'matched_aapos', 'matched_target', 'matched_UKBID', 'matched_index', 'pos_dict',
       'aaref', 'aaalt','Amino_acids',
       'CADD_phred_hg38','CADD_phred_hg19', 'CADDdiff_38minus19', 
       'Cys_reactivity', 'Cys_react_threshold', 'Cys_target_label',
       'Lys_reactivity', 'Lys_react_threshold', 'Lys_target_label', 'rs_dbSNP151', 'Ensembl_transcriptid','cds_strand', 'refcodon', 'codonpos', 'codon_degeneracy','SIFT_score', 'SIFT_converted_rankscore', 'SIFT_pred', 'SIFT4G_score',
       'SIFT4G_converted_rankscore', 'SIFT4G_pred', 'Polyphen2_HDIV_score',
       'Polyphen2_HDIV_rankscore', 'Polyphen2_HDIV_pred',
       'Polyphen2_HVAR_score', 'Polyphen2_HVAR_rankscore',
       'Polyphen2_HVAR_pred','LRT_score', 'LRT_converted_rankscore','LRT_pred', 'LRT_Omega', 'MutationTaster_score','MutationTaster_converted_rankscore', 'MutationTaster_pred','MutationTaster_model', 'MutationTaster_AAE', 'MutationAssessor_score','MutationAssessor_rankscore', 'MutationAssessor_pred',
       'FATHMM_score','FATHMM_converted_rankscore', 'FATHMM_pred', 'PROVEAN_score','PROVEAN_converted_rankscore', 'PROVEAN_pred', 'VEST4_score','VEST4_rankscore', 'MetaSVM_score', 'MetaSVM_rankscore', 'MetaSVM_pred','MetaLR_score', 'MetaLR_rankscore', 'MetaLR_pred', 'Reliability_index','M-CAP_score', 'M-CAP_rankscore', 'M-CAP_pred', 'REVEL_score','REVEL_rankscore', 'MutPred_score', 'MutPred_rankscore',
       'MutPred_protID', 'MutPred_AAchange', 'MutPred_Top5features',
        'MPC_score','MPC_rankscore','PrimateAI_score', 'PrimateAI_rankscore', 'PrimateAI_pred','DANN_score', 'DANN_rankscore', 'fathmm-MKL_coding_score','fathmm-MKL_coding_rankscore', 'fathmm-MKL_coding_pred','fathmm-MKL_coding_group', 'fathmm-XF_coding_score','fathmm-XF_coding_rankscore', 'fathmm-XF_coding_pred','Eigen-raw_coding', 'Eigen-raw_coding_rankscore', 'Eigen-pred_coding',
       'Eigen-PC-raw_coding', 'Eigen-PC-raw_coding_rankscore',
       'Eigen-PC-phred_coding']

simpleD = simpleD[neworder]
simpleD.to_csv("SCORE_dbNSFP_selectcols_detected_CK_104475.csv", index=False)



## not detected dbNSFP score file: 
not_scores = 'SCORE_annotation_3840_CYS_LYS_NOT_allcol_detected.csv'
notscore = pd.read_csv(not_scores, low_memory=False)
print(notscore.shape)
# (1222911, 385)

### simplify df with select columns: 
not_cols = ['pos_id19', 'CADD_phred_hg19', 'CADDdiff_38minus19', 'Amino_acids',
       'pos_ID_falseCKtarget', 'aaref',
       'aaalt', 'rs_dbSNP151', 'genename', 'Ensembl_transcriptid','cds_strand', 'refcodon', 'codonpos','codon_degeneracy', 'SIFT_score', 'SIFT_converted_rankscore', 'SIFT_pred', 'SIFT4G_score',
       'SIFT4G_converted_rankscore', 'SIFT4G_pred', 'Polyphen2_HDIV_score',
       'Polyphen2_HDIV_rankscore', 'Polyphen2_HDIV_pred',
       'Polyphen2_HVAR_score', 'Polyphen2_HVAR_rankscore',
       'Polyphen2_HVAR_pred','LRT_score',
       'LRT_converted_rankscore', 'LRT_pred', 'LRT_Omega',
       'MutationTaster_score', 'MutationTaster_converted_rankscore',
       'MutationTaster_pred', 'MutationTaster_model', 'MutationTaster_AAE',
       'MutationAssessor_score', 'MutationAssessor_rankscore',
       'MutationAssessor_pred', 'FATHMM_score', 'FATHMM_converted_rankscore',
       'FATHMM_pred', 'PROVEAN_score', 'PROVEAN_converted_rankscore',
       'PROVEAN_pred', 'VEST4_score', 'VEST4_rankscore', 'MetaSVM_score',
       'MetaSVM_rankscore', 'MetaSVM_pred', 'MetaLR_score', 'MetaLR_rankscore',
       'MetaLR_pred', 'Reliability_index', 'M-CAP_score', 'M-CAP_rankscore',
       'M-CAP_pred', 'REVEL_score', 'REVEL_rankscore', 'MutPred_score',
       'MutPred_rankscore', 'MutPred_protID', 'MutPred_AAchange',
       'MutPred_Top5features', 'MPC_score','MPC_rankscore', 'PrimateAI_score', 'PrimateAI_rankscore',
       'PrimateAI_pred','CADD_phred','DANN_score', 'DANN_rankscore', 'fathmm-MKL_coding_score',
       'fathmm-MKL_coding_rankscore', 'fathmm-MKL_coding_pred',
       'fathmm-MKL_coding_group', 'fathmm-XF_coding_score',
       'fathmm-XF_coding_rankscore', 'fathmm-XF_coding_pred',
       'Eigen-raw_coding', 'Eigen-raw_coding_rankscore', 'Eigen-pred_coding',
       'Eigen-PC-raw_coding', 'Eigen-PC-raw_coding_rankscore',
       'Eigen-PC-phred_coding', 'matched_UKBID',
       'matched_aapos', 'matched_index', 'pos_id38']
# filter
simpleN = notscore[not_cols].copy()
# rename
simpleN.columns = ['pos_id19', 'CADD_phred_hg19', 'CADDdiff_38minus19', 'Amino_acids',
       'pos_ID', 'aaref',
       'aaalt', 'rs_dbSNP151', 'genename', 'Ensembl_transcriptid','cds_strand', 'refcodon', 'codonpos','codon_degeneracy', 'SIFT_score', 'SIFT_converted_rankscore', 'SIFT_pred', 'SIFT4G_score',
       'SIFT4G_converted_rankscore', 'SIFT4G_pred', 'Polyphen2_HDIV_score',
       'Polyphen2_HDIV_rankscore', 'Polyphen2_HDIV_pred',
       'Polyphen2_HVAR_score', 'Polyphen2_HVAR_rankscore',
       'Polyphen2_HVAR_pred','LRT_score',
       'LRT_converted_rankscore', 'LRT_pred', 'LRT_Omega',
       'MutationTaster_score', 'MutationTaster_converted_rankscore',
       'MutationTaster_pred', 'MutationTaster_model', 'MutationTaster_AAE',
       'MutationAssessor_score', 'MutationAssessor_rankscore',
       'MutationAssessor_pred', 'FATHMM_score', 'FATHMM_converted_rankscore',
       'FATHMM_pred', 'PROVEAN_score', 'PROVEAN_converted_rankscore',
       'PROVEAN_pred', 'VEST4_score', 'VEST4_rankscore', 'MetaSVM_score',
       'MetaSVM_rankscore', 'MetaSVM_pred', 'MetaLR_score', 'MetaLR_rankscore',
       'MetaLR_pred', 'Reliability_index', 'M-CAP_score', 'M-CAP_rankscore',
       'M-CAP_pred', 'REVEL_score', 'REVEL_rankscore', 'MutPred_score',
       'MutPred_rankscore', 'MutPred_protID', 'MutPred_AAchange',
       'MutPred_Top5features', 'MPC_score','MPC_rankscore', 'PrimateAI_score', 'PrimateAI_rankscore',
       'PrimateAI_pred','CADD_phred_hg38','DANN_score', 'DANN_rankscore', 'fathmm-MKL_coding_score',
       'fathmm-MKL_coding_rankscore', 'fathmm-MKL_coding_pred',
       'fathmm-MKL_coding_group', 'fathmm-XF_coding_score',
       'fathmm-XF_coding_rankscore', 'fathmm-XF_coding_pred',
       'Eigen-raw_coding', 'Eigen-raw_coding_rankscore', 'Eigen-pred_coding',
       'Eigen-PC-raw_coding', 'Eigen-PC-raw_coding_rankscore',
       'Eigen-PC-phred_coding', 'matched_UKBID',
       'matched_aapos', 'matched_index', 'pos_id38']
       
# change order
neworder = ['pos_id19', 'pos_id38', 'pos_ID', 'matched_UKBID', 'matched_aapos', 'matched_index','Ensembl_transcriptid', 'Amino_acids','aaref','aaalt', 
'CADD_phred_hg38', 'CADD_phred_hg19', 'CADDdiff_38minus19', 'rs_dbSNP151', 'genename', 'cds_strand', 'refcodon', 'codonpos','codon_degeneracy', 'SIFT_score', 'SIFT_converted_rankscore', 'SIFT_pred', 'SIFT4G_score',
       'SIFT4G_converted_rankscore', 'SIFT4G_pred', 'Polyphen2_HDIV_score',
       'Polyphen2_HDIV_rankscore', 'Polyphen2_HDIV_pred',
       'Polyphen2_HVAR_score', 'Polyphen2_HVAR_rankscore',
       'Polyphen2_HVAR_pred','LRT_score',
       'LRT_converted_rankscore', 'LRT_pred', 'LRT_Omega',
       'MutationTaster_score', 'MutationTaster_converted_rankscore',
       'MutationTaster_pred', 'MutationTaster_model', 'MutationTaster_AAE',
       'MutationAssessor_score', 'MutationAssessor_rankscore',
       'MutationAssessor_pred', 'FATHMM_score', 'FATHMM_converted_rankscore',
       'FATHMM_pred', 'PROVEAN_score', 'PROVEAN_converted_rankscore',
       'PROVEAN_pred', 'VEST4_score', 'VEST4_rankscore', 'MetaSVM_score',
       'MetaSVM_rankscore', 'MetaSVM_pred', 'MetaLR_score', 'MetaLR_rankscore',
       'MetaLR_pred', 'Reliability_index', 'M-CAP_score', 'M-CAP_rankscore',
       'M-CAP_pred', 'REVEL_score', 'REVEL_rankscore', 'MutPred_score',
       'MutPred_rankscore', 'MutPred_protID', 'MutPred_AAchange',
       'MutPred_Top5features', 'MPC_score','MPC_rankscore', 'PrimateAI_score', 'PrimateAI_rankscore',
       'PrimateAI_pred','DANN_score', 'DANN_rankscore', 'fathmm-MKL_coding_score',
       'fathmm-MKL_coding_rankscore', 'fathmm-MKL_coding_pred',
       'fathmm-MKL_coding_group', 'fathmm-XF_coding_score',
       'fathmm-XF_coding_rankscore', 'fathmm-XF_coding_pred',
       'Eigen-raw_coding', 'Eigen-raw_coding_rankscore', 'Eigen-pred_coding',
       'Eigen-PC-raw_coding', 'Eigen-PC-raw_coding_rankscore',
       'Eigen-PC-phred_coding']
simpleN = simpleN[neworder]
simpleN.to_csv("SCORE_dbNSFP_selectcols_NOT_detected_CK_1222911.csv", index=False)
```

```python 
header = simpleD.columns
header2 = simpleN.columns
d = {header[i] : i for i in range(0, len(header))}
# print(d) # 'matched_index': 7, 'Ensembl_transcriptid': 8,
d2 = {header2[i] : i for i in range(0, len(header2))}
# print(d2) # 'matched_index': 5, 'Ensembl_transcriptid': 6, 
```


## columns parsed: 
```
{DETECTED
 'matched_index': 6,
 'Ensembl_transcriptid': 21,
 'SIFT_score': 26,
 'SIFT_pred': 28,
 'SIFT4G_score': 29,
 'SIFT4G_pred': 31,
 'Polyphen2_HDIV_score': 32,
 'Polyphen2_HDIV_pred': 34,
 'Polyphen2_HVAR_score': 35,
 'Polyphen2_HVAR_pred': 37,
 'MutationTaster_score': 42,
 'MutationTaster_pred': 44,
 'MutationTaster_model': 45,
 'MutationTaster_AAE': 46,
 'MutationAssessor_score': 47,
 'MutationAssessor_pred': 49,
 'FATHMM_score': 50,
 'FATHMM_pred': 52,
 'PROVEAN_score': 53,
 'PROVEAN_pred': 55,
 'VEST4_score': 56,
 'MutPred_Top5features': 74,
 'MPC_score': 75,
 }

NOT DETECTED
{
 'matched_index': 5,
 'Ensembl_transcriptid': 6,
 'SIFT_score': 19,
 'SIFT_pred': 21,
 'SIFT4G_score': 22,
 'SIFT4G_pred': 24,
 'Polyphen2_HDIV_score': 25,
 'Polyphen2_HDIV_pred': 27,
 'Polyphen2_HVAR_score': 28,
 'Polyphen2_HVAR_pred': 30,
 'MutationTaster_score': 35,
 'MutationTaster_pred': 37,
 'MutationTaster_model': 38,
 'MutationTaster_AAE': 39,
 'MutationAssessor_score': 40,
 'MutationAssessor_pred': 42,
 'FATHMM_score': 43,
 'FATHMM_pred': 45,
 'PROVEAN_score': 46,
 'PROVEAN_pred': 48,
 'VEST4_score': 49,
 'MutPred_Top5features': 67,
 'MPC_score': 68,
}
```

# (done)  parsing list values in columns of dbNSFP files, 
```python

def feature_id_col(filename, outfile, subset):
    if subset == 'dect':
        # adding feature_ID column from parsing enst list col from dbNSFP with matched_index col
        # modeled after function from Pmap_parseID_correction.py
        with open(filename, newline='') as file:
            # read in file, save header
            csvReader = csv.reader(file)
            header = next(csvReader)
            # create and write to outfile
            os.system("touch %s" % (outfile))
            with open(outfile, 'w') as out:
                csvWriter = csv.writer(out)
                csvWriter.writerow(header)
            # loop over rows
            for row in csvReader:
                matchI = int(row[6])
                changeindex = [21,26,28,29,31,32,34,35,37,42,44,45,46,47,49,50,52,53,55,56,74,75]
                for coli in changeindex:
                    rowvalue = row[coli]
                    splitval = rowvalue.split(";")
                    if matchI < len(splitval):
                        newval = splitval[matchI]
                        row[coli] = newval
                with open(outfile, 'a') as out:
                        csvWriter = csv.writer(out)
                        csvWriter.writerow(row)
        print("done with : ", outfile)
    if subset == 'notdect':
        # adding feature_ID column from parsing enst list col from dbNSFP with matched_index col
        # modeled after function from Pmap_parseID_correction.py
        with open(filename, newline='') as file:
            # read in file, save header
            csvReader = csv.reader(file)
            header = next(csvReader)
            # create and write to outfile
            os.system("touch %s" % (outfile))
            with open(outfile, 'w') as out:
                csvWriter = csv.writer(out)
                csvWriter.writerow(header)
            # loop over rows
            for row in csvReader:
                matchI = int(row[5])
                changeindex = [6,19,21,22,24,25,27,28,30,35,37,38,39,40,42,43,45,46,48,49,67,68]
                for coli in changeindex:
                    rowvalue = row[coli]
                    splitval = rowvalue.split(";")
                    if matchI < len(splitval):
                        newval = splitval[matchI]
                        row[coli] = newval
                with open(outfile, 'a') as out:
                        csvWriter = csv.writer(out)
                        csvWriter.writerow(row)
        print("done with : ", outfile)
        
        
# calling function
feature_id_col('SCORE_dbNSFP_selectcols_detected_CK_104475.csv', 'SCORE2_dbNSFP_parsedcols_detected_CK_104475.csv', 'dect')
feature_id_col('SCORE_dbNSFP_selectcols_NOT_detected_CK_1222911.csv', 'SCORE2_dbNSFP_parsedcols_NOT_detected_CK_1222911.csv', 'notdect')
```

## result of feature_id_col() =
- all dbnsfp file rows have 1 ENST ID matching index of canonical ukb ID

description files for new featureID column added created with below code:
```python
saveColumnValues(dbnsfp, 'FeatureID', 'dbnsfp_featureID_detected_description.csv')
saveColumnValues(notdbnsfp, 'FeatureID', 'dbnsfp_featureID_notdetected_description.csv')
```



In [1]:
import os
import sys
import argparse
import pandas as pd
import csv
import numpy as np

from IPython.display import display, HTML
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000
display(HTML("<style>.container {width:90% !important;}</style>"))
sys.path.append("/Users/mariapalafox/Desktop/Toolbox")
from all_funx import *
from maplib import *

# [5] MERGE CADD chr files with new pos_id column with dbNSFP column of pos_id19 or pos_id38 for DETECTED and NONDETECTED positions

## 5a. MERGE DETECTED 37 CADD ALL CONSEQUENCE with pos_ids from dbNSFP

In [13]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import pandas as pd

# Pmap_dbNSFP_CADD_merge_DECT19.py
'''
filter files: 

col_det_posid19.csv --- ALLCON_{}_CADD_GRCh37_DETECTED_CK.csv

'''

def main():
    os.chdir('/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CADDmapped/RESULT_CADDv14_pos_overlap_dbNSFPcoordinates/ALL_CONSEQUENCES/37det/')

    chrls = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X','Y']

    for order in chrls:
        chrID = 'chr{}'.format(order)
        file1 = 'ALLCON_{}_CADD_GRCh37_DETECTED_CK.csv'.format(chrID)
        out1 = 'FILTERONposid_{}_CADD_GRCh37_DETECTED_CK.csv'.format(chrID)
        df1 = pd.read_csv(file1, low_memory=False)

        # filter file
        fildf = pd.read_csv('col_det_posid19.csv')

        print("filter file shape: ")
        print(fildf.shape)
        print("chr cadd file shape: ")
        print(df1.shape)

        dropme = ['Type', 'Length', 'AnnoType','ConsScore', 'ConsDetail','oAA', 'nAA', 'GeneID','FeatureID', 'GeneName', 'CCDS', 'Intron', 'Exon', 'cDNApos','relcDNApos', 'CDSpos', 'relCDSpos', 'relProtPos', 'Domain','Dst2Splice', 'Dst2SplType', 'minDistTSS', 'minDistTSE']

        df1.drop(dropme, axis=1, inplace=True)
        df1.drop_duplicates(keep='first', inplace = True)

        mer = pd.merge(df1, fildf, how='inner', on=['pos_id19'])
        print("merge shape : ")
        print(mer.shape)
        mer.to_csv(out1, index=False)
        print("unique pos_id: ")
        print(len(set(mer.pos_id19)))
        print("done with ", order)
main()

filter file shape: 
(104475, 1)
chr cadd file shape: 
(22764, 65)
merge shape : 
(16832, 42)
unique pos_id: 
10556
done with  1
filter file shape: 
(104475, 1)
chr cadd file shape: 
(16746, 65)
merge shape : 
(12556, 42)
unique pos_id: 
7693
done with  2
filter file shape: 
(104475, 1)
chr cadd file shape: 
(11871, 65)
merge shape : 
(8449, 42)
unique pos_id: 
5110
done with  3
filter file shape: 
(104475, 1)
chr cadd file shape: 
(8217, 65)
merge shape : 
(6252, 42)
unique pos_id: 
3878
done with  4
filter file shape: 
(104475, 1)
chr cadd file shape: 
(9306, 65)
merge shape : 
(6685, 42)
unique pos_id: 
4249
done with  5
filter file shape: 
(104475, 1)
chr cadd file shape: 
(12342, 65)
merge shape : 
(8799, 42)
unique pos_id: 
4501
done with  6
filter file shape: 
(104475, 1)
chr cadd file shape: 
(12852, 65)
merge shape : 
(9128, 42)
unique pos_id: 
5460
done with  7
filter file shape: 
(104475, 1)
chr cadd file shape: 
(6879, 65)
merge shape : 
(5121, 42)
unique pos_id: 
3269
done 

## 5b. MERGE DETECTED 38 CADD ALL CONSEQUENCE with pos_ids from dbNSFP

In [11]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import pandas as pd

# Pmap_dbNSFP_CADD_merge_DECT19.py
'''
filter files: 

col_det_posid38.csv --- ALLCON_{}_CADD_GRCh38_DETECTED_CK.csv

'''

def main():
    # CHANGE PATH HERE
    os.chdir('/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CADDmapped/RESULT_CADDv14_pos_overlap_dbNSFPcoordinates/ALL_CONSEQUENCES/38det/')

    chrls = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X','Y']

    for order in chrls:
        chrID = 'chr{}'.format(order)
        
        # CHANGE FILE NAMES HERE
        file1 = 'ALLCON_{}_CADD_GRCh38_DETECTED_CK.csv'.format(chrID)
        out1 = 'FILTERONposid_{}_CADD_GRCh38_DETECTED_CK.csv'.format(chrID)
        df1 = pd.read_csv(file1, low_memory=False)

        # CHANGE FILTER FILE NAME HERE
        fildf = pd.read_csv('col_det_posid38.csv')

        print("filter file shape: ")
        print(fildf.shape)
        print("chr cadd file shape: ")
        print(df1.shape)

        dropme = ['Type', 'Length', 'AnnoType','ConsScore', 'ConsDetail','oAA', 'nAA', 'GeneID','FeatureID', 'GeneName', 'CCDS', 'Intron', 'Exon', 'cDNApos','relcDNApos', 'CDSpos', 'relCDSpos', 'relProtPos', 'Domain','Dst2Splice', 'Dst2SplType', 'minDistTSS', 'minDistTSE']

        df1.drop(dropme, axis=1, inplace=True)
        df1.drop_duplicates(keep='first', inplace = True)

        mer = pd.merge(df1, fildf, how='inner', on=['pos_id38'])
        print("merge shape : ")
        print(mer.shape)
        mer.to_csv(out1, index=False)
        print("unique pos_id: ")
        print(len(set(mer.pos_id38)))
        print("done with ", order)
        print()
main()

filter file shape: 
(104475, 1)
chr cadd file shape: 
(23163, 65)
merge shape : 
(17093, 42)
done with  1
filter file shape: 
(104475, 1)
chr cadd file shape: 
(16806, 65)
merge shape : 
(12469, 42)
done with  2
filter file shape: 
(104475, 1)
chr cadd file shape: 
(11985, 65)
merge shape : 
(8510, 42)
done with  3
filter file shape: 
(104475, 1)
chr cadd file shape: 
(8145, 65)
merge shape : 
(6188, 42)
done with  4
filter file shape: 
(104475, 1)
chr cadd file shape: 
(9522, 65)
merge shape : 
(6864, 42)
done with  5
filter file shape: 
(104475, 1)
chr cadd file shape: 
(12711, 65)
merge shape : 
(8974, 42)
done with  6
filter file shape: 
(104475, 1)
chr cadd file shape: 
(13095, 65)
merge shape : 
(9289, 42)
done with  7
filter file shape: 
(104475, 1)
chr cadd file shape: 
(7620, 65)
merge shape : 
(5638, 42)
done with  8
filter file shape: 
(104475, 1)
chr cadd file shape: 
(8772, 65)
merge shape : 
(6333, 42)
done with  9
filter file shape: 
(104475, 1)
chr cadd file shape: 
(95

## 5c. MERGE NOT DETECTED 37 CADD ALL CONSEQUENCE with pos_ids from dbNSFP

In [14]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import pandas as pd

# Pmap_dbNSFP_CADD_merge_DECT19.py
'''
filter files: 

col_notdet_posid19.csv --- ALLCON_{}_CADD_GRCh37_NOT_DETECTED_CK.csv

'''

def main():
    os.chdir('/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CADDmapped/RESULT_CADDv14_pos_overlap_dbNSFPcoordinates/ALL_CONSEQUENCES/37not/')

    chrls = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X','Y']

    for order in chrls:
        chrID = 'chr{}'.format(order)
        file1 = 'ALLCON_{}_CADD_GRCh37_NOT_DETECTED_CK.csv'.format(chrID)
        out1 = 'FILTERONposid_{}_CADD_GRCh37_NOT_DETECTED_CK.csv'.format(chrID)
        df1 = pd.read_csv(file1, low_memory=False)

        # filter file
        fildf = pd.read_csv('col_notdet_posid19.csv')

        print("filter file shape: ")
        print(fildf.shape)
        print("chr cadd file shape: ")
        print(df1.shape)

        dropme = ['Type', 'Length', 'AnnoType','ConsScore', 'ConsDetail','oAA', 'nAA', 'GeneID','FeatureID', 'GeneName', 'CCDS', 'Intron', 'Exon', 'cDNApos','relcDNApos', 'CDSpos', 'relCDSpos', 'relProtPos', 'Domain','Dst2Splice', 'Dst2SplType', 'minDistTSS', 'minDistTSE']

        df1.drop(dropme, axis=1, inplace=True)
        df1.drop_duplicates(keep='first', inplace = True)

        mer = pd.merge(df1, fildf, how='inner', on=['pos_id19'])
        print("merge shape : ")
        print(mer.shape)
        mer.to_csv(out1, index=False)
        print("done with ", order)
main()

filter file shape: 
(1222911, 1)
chr cadd file shape: 
(219981, 65)
merge shape : 
(165555, 42)
done with  1
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(189372, 65)
merge shape : 
(144152, 42)
done with  2
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(146550, 65)
merge shape : 
(107666, 42)
done with  3
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(78240, 65)
merge shape : 
(60230, 42)
done with  4
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(125283, 65)
merge shape : 
(93395, 42)
done with  5
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(132042, 65)
merge shape : 
(98700, 42)
done with  6
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(105234, 65)
merge shape : 
(77000, 42)
done with  7
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(75387, 65)
merge shape : 
(55634, 42)
done with  8
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(88968, 65)
merge shape : 
(66091, 42)
done with  9
filter file shape: 
(1222911

## 5d. MERGE NOT DETECTED 38 CADD ALL CONSEQUENCE with pos_ids from dbNSFP

In [12]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import pandas as pd

# Pmap_dbNSFP_CADD_merge_DECT19.py
'''
filter files: 

col_notdet_posid38.csv --- ALLCON_{}_CADD_GRCh38_NOT_DETECTED_CK.csv

'''

def main():
    # CHANGE PATH HERE
    os.chdir('/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CADDmapped/RESULT_CADDv14_pos_overlap_dbNSFPcoordinates/ALL_CONSEQUENCES/38not/')

    chrls = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X','Y']

    for order in chrls:
        chrID = 'chr{}'.format(order)
        
        # CHANGE FILE NAMES HERE
        file1 = 'ALLCON_{}_CADD_GRCh38_NOT_DETECTED_CK.csv'.format(chrID)
        out1 = 'FILTERONposid_{}_CADD_GRCh38_NOT_DETECTED_CK.csv'.format(chrID)
        df1 = pd.read_csv(file1, low_memory=False)

        # CHANGE FILTER FILE NAME HERE
        fildf = pd.read_csv('col_notdet_posid38.csv')

        print("filter file shape: ")
        print(fildf.shape)
        print("chr cadd file shape: ")
        print(df1.shape)

        dropme = ['Type', 'Length', 'AnnoType','ConsScore', 'ConsDetail','oAA', 'nAA', 'GeneID','FeatureID', 'GeneName', 'CCDS', 'Intron', 'Exon', 'cDNApos','relcDNApos', 'CDSpos', 'relCDSpos', 'relProtPos', 'Domain','Dst2Splice', 'Dst2SplType', 'minDistTSS', 'minDistTSE']

        df1.drop(dropme, axis=1, inplace=True)
        df1.drop_duplicates(keep='first', inplace = True)

        mer = pd.merge(df1, fildf, how='inner', on=['pos_id38'])
        print("merge shape : ")
        print(mer.shape)
        mer.to_csv(out1, index=False)
        print("done with ", order)
main()


filter file shape: 
(1222911, 1)
chr cadd file shape: 
(220821, 65)
merge shape : 
(166751, 42)
done with  1
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(188106, 65)
merge shape : 
(143637, 42)
done with  2
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(147309, 65)
merge shape : 
(107811, 42)
done with  3
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(78663, 65)
merge shape : 
(60484, 42)
done with  4
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(128820, 65)
merge shape : 
(96473, 42)
done with  5
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(135543, 65)
merge shape : 
(101513, 42)
done with  6
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(106776, 65)
merge shape : 
(78054, 42)
done with  7
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(78834, 65)
merge shape : 
(57927, 42)
done with  8
filter file shape: 
(1222911, 1)
chr cadd file shape: 
(90018, 65)
merge shape : 
(67467, 42)
done with  9
filter file shape: 
(122291

# [6] merge CADD chr files

```bash
#!/bin/bash
# keeps header and appends all rows together
# for Pmap dbNSFP CADD mapped detected and non detected chr merge

awk -F ',' 'FNR==1 && NR!=1 { while (/^pos_hg19/) getline; } 1 {print}' FILTERONposid_chr*_CADD_GRCh37_DETECTED_CK.csv > SCORE_FILTERallcon_CADD_37_DETECTED.csv

awk -F ',' 'FNR==1 && NR!=1 { while (/^pos_hg19/) getline; } 1 {print}' FILTERONposid_chr*_CADD_GRCh37_NOT_DETECTED_CK.csv >SCORE_FILTERallcon_CADD_37_NOT_DETECTED.csv

awk -F ',' 'FNR==1 && NR!=1 { while (/^pos_hg38/) getline; } 1 {print}' FILTERONposid_chr*_CADD_GRCh38_DETECTED_CK.csv > SCORE_FILTERallcon_CADD_38_DETECTED.csv

awk -F ',' 'FNR==1 && NR!=1 { while (/^pos_hg38/) getline; } 1 {print}' FILTERONposid_chr*_CADD_GRCh38_NOT_DETECTED_CK.csv >SCORE_FILTERallcon_CADD_38_NOT_DETECTED.csv
```

# [6a] prep for dbNSFP merge, pos_id19 or pos_id38 in CADD files should all be unique
- make nonredundant rows by dropping columns that are not needed and then dropping duplicate rows

In [2]:
os.chdir('/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CADDmapped/RESULT_CADDv14_pos_overlap_dbNSFPcoordinates/ALL_CONSEQUENCES/')

In [3]:
not37 = pd.read_csv("SCORE_FILTERallcon_CADD_37_NOT_DETECTED.csv", low_memory=False)
not38 = pd.read_csv("SCORE_FILTERallcon_CADD_38_NOT_DETECTED.csv", low_memory=False)
det37 = pd.read_csv("SCORE_FILTERallcon_CADD_37_DETECTED.csv", low_memory=False)
det38 = pd.read_csv("SCORE_FILTERallcon_CADD_38_DETECTED.csv", low_memory=False)

In [4]:
# POS ID unique count check
dfls = [det37, not37]
for i in dfls:
    print(len(list(i.pos_id19)))
    print(len(set(i.pos_id19)))
    print()
    
dfls = [det38, not38]
for i in dfls:
    print(len(list(i.pos_id38)))
    print(len(set(i.pos_id38)))
    print()
    
print("Unique number of uniprot and AA IDs, equivalent to everlabeled counts for detected")
dfls = [det37, not37, det38, not38]
for i in dfls:
    print(len(set(i.pos_ID)))
    print()

177852
104475

1827947
1222911

182111
104475

1860738
1222911

Unique number of uniprot and AA IDs, equivalent to everlabeled counts for detected
14925

174702

14925

174702



**MATCHES ACCOUNTING #'s for dbNSFP pos_id...therefore ALL dbNSFP pos are in CADD files, now filter for only neccessary columns, drop dup rows, and save**

| Unique column counts | DETECTED pos_ids | NOT DETECTED pos_ids |
|---------|------------|-----------|
| pos_id19 or pos_id38 | 104,475 | 1,222,911 | 
| pos_ID ukbAA | 14,925 | 174,702 | 


In [19]:
det37.columns

Index(['pos_hg19', 'chr', 'pos_ID', 'Ref', 'Alt', 'Consequence', 'GC', 'CpG',
       'motifECount', 'motifEName', 'motifEHIPos', 'motifEScoreChng',
       'protPos', 'SIFTcat', 'SIFTval', 'PolyPhenCat', 'PolyPhenVal',
       'priPhCons', 'mamPhCons', 'verPhCons', 'priPhyloP', 'mamPhyloP',
       'verPhyloP', 'GerpRS', 'GerpRSpval', 'GerpN', 'GerpS', 'Grantham',
       'Dist2Mutation', 'Freq100bp', 'Rare100bp', 'Sngl100bp', 'Freq1000bp',
       'Rare1000bp', 'Sngl1000bp', 'Freq10000bp', 'Rare10000bp', 'Sngl10000bp',
       'RawScore', 'PHRED', 'pos_id19', 'Amino_acids'],
      dtype='object')

## Dropping CADD columns that may casuse redundancy (multimapping of pos_id's). Columns of interest are only related to genomic coordinates so differences in missense annotation do not matter

**goal is CADD files with non redundant pos_id column to merge with dbNSFP annotations, 1:1 merge**

In [5]:
dropme = ['pos_hg19','chr', 'pos_ID', 'Ref', 'Alt', 'Consequence', 'motifECount', 'motifEName', 'motifEHIPos', 'motifEScoreChng',
       'protPos', 'SIFTcat', 'SIFTval', 'PolyPhenCat', 'PolyPhenVal','Grantham', 'Amino_acids']

dfls = [det37, not37]
for i in dfls:
    i.drop(dropme, axis=1, inplace=True)
    print(i.shape)
    i.drop_duplicates(keep='first', inplace = True)
    print(i.shape)
    print()

(177852, 25)
(104475, 25)

(1827947, 25)
(1222911, 25)



In [6]:
dropme = ['pos_hg38','chr', 'pos_ID', 'Ref', 'Alt', 'Consequence', 'motifECount', 'motifEName', 'motifEHIPos', 'motifEScoreChng',
       'protPos', 'SIFTcat', 'SIFTval', 'PolyPhenCat', 'PolyPhenVal','Grantham', 'Amino_acids']

dfls = [det38, not38]
for i in dfls:
    i.drop(dropme, axis=1, inplace=True)
    print(i.shape)
    i.drop_duplicates(keep='first', inplace = True)
    print(i.shape)
    print()

(182111, 25)
(104475, 25)

(1860738, 25)
(1222911, 25)



In [7]:
# prep col names for merging with dbNSFP file
det37.columns = ['GC_hg19', 'CpG_hg19', 'priPhCons_hg19', 'mamPhCons_hg19', 'verPhCons_hg19',
       'priPhyloP_hg19', 'mamPhyloP_hg19', 'verPhyloP_hg19', 'GerpRS_hg19', 'GerpRSpval_hg19', 'GerpN_hg19',
       'GerpS_hg19', 'Dist2Mutation_hg19', 'Freq100bp_hg19', 'Rare100bp_hg19', 'Sngl100bp_hg19',
       'Freq1000bp_hg19', 'Rare1000bp_hg19', 'Sngl1000bp_hg19', 'Freq10000bp_hg19', 'Rare10000bp_hg19',
       'Sngl10000bp_hg19', 'RawScore_hg19', 'PHRED_hg19', 'pos_id19']

In [8]:
det38.columns = ['GC_hg38', 'CpG_hg38', 'priPhCons_hg38', 'mamPhCons_hg38', 'verPhCons_hg38', 'priPhyloP_hg38','mamPhyloP_hg38', 'verPhyloP_hg38', 'GerpRS_hg38', 'GerpRSpval_hg38', 'GerpN_hg38', 'GerpS_hg38','Dist2Mutation_hg38', 'Freq100bp_hg38', 'Rare100bp_hg38', 'Sngl100bp_hg38', 'Freq1000bp_hg38','Rare1000bp_hg38', 'Sngl1000bp_hg38', 'Freq10000bp_hg38', 'Rare10000bp_hg38', 'Sngl10000bp_hg38','RawScore_hg38', 'PHRED_hg38', 'pos_id38']

In [9]:
not37.columns = ['GC_hg19', 'CpG_hg19', 'priPhCons_hg19', 'mamPhCons_hg19', 'verPhCons_hg19', 'priPhyloP_hg19','mamPhyloP_hg19', 'verPhyloP_hg19', 'GerpRS_hg19', 'GerpRSpval_hg19', 'GerpN_hg19', 'GerpS_hg19','Dist2Mutation_hg19', 'Freq100bp_hg19', 'Rare100bp_hg19', 'Sngl100bp_hg19', 'Freq1000bp_hg19','Rare1000bp_hg19', 'Sngl1000bp_hg19', 'Freq10000bp_hg19', 'Rare10000bp_hg19', 'Sngl10000bp_hg19','RawScore_hg19', 'PHRED_hg19', 'pos_id19']

In [10]:
not38.columns = ['GC_hg38', 'CpG_hg38', 'priPhCons_hg38', 'mamPhCons_hg38', 'verPhCons_hg38', 'priPhyloP_hg38', 'mamPhyloP_hg38', 'verPhyloP_hg38', 'GerpRS_hg38', 'GerpRSpval_hg38', 'GerpN_hg38', 'GerpS_hg38','Dist2Mutation_hg38', 'Freq100bp_hg38', 'Rare100bp_hg38', 'Sngl100bp_hg38', 'Freq1000bp_hg38', 'Rare1000bp_hg38', 'Sngl1000bp_hg38', 'Freq10000bp_hg38', 'Rare10000bp_hg38', 'Sngl10000bp_hg38','RawScore_hg38', 'PHRED_hg38', 'pos_id38']

In [11]:
det37.head(3)

,GC_hg19,CpG_hg19,priPhCons_hg19,mamPhCons_hg19,verPhCons_hg19,priPhyloP_hg19,mamPhyloP_hg19,verPhyloP_hg19,GerpRS_hg19,GerpRSpval_hg19,GerpN_hg19,GerpS_hg19,Dist2Mutation_hg19,Freq100bp_hg19,Rare100bp_hg19,Sngl100bp_hg19,Freq1000bp_hg19,Rare1000bp_hg19,Sngl1000bp_hg19,Freq10000bp_hg19,Rare10000bp_hg19,Sngl10000bp_hg19,RawScore_hg19,PHRED_hg19,pos_id19
0,0.470199,0.053333,0.084,0.084,1.0,0.094,0.095,0.877,NaN,NaN,0.0,0.0,38936.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,24.0,137.0,2.381671,22.30,10_000093270_A_C
2,0.463576,0.053333,0.083,0.082,1.0,0.110,0.109,3.127,NaN,NaN,0.0,0.0,38936.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,24.0,137.0,2.267094,21.80,10_000093271_C_A
4,0.463576,0.053333,0.083,0.082,1.0,0.110,0.109,3.127,NaN,NaN,0.0,0.0,38936.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,24.0,137.0,2.045093,19.67,10_000093271_C_G


In [12]:
not37.head(3)

,GC_hg19,CpG_hg19,priPhCons_hg19,mamPhCons_hg19,verPhCons_hg19,priPhyloP_hg19,mamPhyloP_hg19,verPhyloP_hg19,GerpRS_hg19,GerpRSpval_hg19,GerpN_hg19,GerpS_hg19,Dist2Mutation_hg19,Freq100bp_hg19,Rare100bp_hg19,Sngl100bp_hg19,Freq1000bp_hg19,Rare1000bp_hg19,Sngl1000bp_hg19,Freq10000bp_hg19,Rare10000bp_hg19,Sngl10000bp_hg19,RawScore_hg19,PHRED_hg19,pos_id19
0,0.536424,0.106667,0.550,0.546,1.0,0.110,0.109,1.700,NaN,NaN,0.0,0.0,38936.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,24.0,137.0,2.636298,22.9,10_000093156_C_A
2,0.536424,0.106667,0.550,0.546,1.0,0.110,0.109,1.700,NaN,NaN,0.0,0.0,38936.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,24.0,137.0,2.627006,22.8,10_000093156_C_G
4,0.529801,0.106667,0.546,0.542,1.0,0.094,0.095,2.564,NaN,NaN,0.0,0.0,38936.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,24.0,137.0,2.819085,23.2,10_000093157_T_A


## importing dbNSFP files with select/parse column values:

In [13]:
detdbnsfp = pd.read_csv("SCORE2_dbNSFP_parsedcols_detected_CK_104475.csv", low_memory=False)

notdbnsfp = pd.read_csv("SCORE2_dbNSFP_parsedcols_NOT_detected_CK_1222911.csv", low_memory=False)

In [14]:
detdbnsfp.head(3)

,pos_id19,pos_id38,pos_ID,matched_aapos,matched_target,matched_UKBID,matched_index,pos_dict,aaref,aaalt,Amino_acids,CADD_phred_hg38,CADD_phred_hg19,CADDdiff_38minus19,Cys_reactivity,Cys_react_threshold,Cys_target_label,Lys_reactivity,Lys_react_threshold,Lys_target_label,rs_dbSNP151,Ensembl_transcriptid,cds_strand,refcodon,codonpos,codon_degeneracy,SIFT_score,SIFT_converted_rankscore,SIFT_pred,SIFT4G_score,SIFT4G_converted_rankscore,SIFT4G_pred,Polyphen2_HDIV_score,Polyphen2_HDIV_rankscore,Polyphen2_HDIV_pred,Polyphen2_HVAR_score,Polyphen2_HVAR_rankscore,Polyphen2_HVAR_pred,LRT_score,LRT_converted_rankscore,LRT_pred,LRT_Omega,MutationTaster_score,MutationTaster_converted_rankscore,MutationTaster_pred,MutationTaster_model,MutationTaster_AAE,MutationAssessor_score,MutationAssessor_rankscore,MutationAssessor_pred,FATHMM_score,FATHMM_converted_rankscore,FATHMM_pred,PROVEAN_score,PROVEAN_converted_rankscore,PROVEAN_pred,VEST4_score,VEST4_rankscore,MetaSVM_score,MetaSVM_rankscore,MetaSVM_pred,MetaLR_score,MetaLR_rankscore,MetaLR_pred,Reliability_index,M-CAP_score,M-CAP_rankscore,M-CAP_pred,REVEL_score,REVEL_rankscore,MutPred_score,MutPred_rankscore,MutPred_protID,MutPred_AAchange,MutPred_Top5features,MPC_score,MPC_rankscore,PrimateAI_score,PrimateAI_rankscore,PrimateAI_pred,DANN_score,DANN_rankscore,fathmm-MKL_coding_score,fathmm-MKL_coding_rankscore,fathmm-MKL_coding_pred,fathmm-MKL_coding_group,fathmm-XF_coding_score,fathmm-XF_coding_rankscore,fathmm-XF_coding_pred,Eigen-raw_coding,Eigen-raw_coding_rankscore,Eigen-pred_coding,Eigen-PC-raw_coding,Eigen-PC-raw_coding_rankscore,Eigen-PC-phred_coding
0,10_000093270_A_C,10_000047330_A_C,Q3ZCM7_C354,354,C354,Q3ZCM7,2,"{12: 'C', 239: 'C', 303: 'C', 354: 'C'}",C,W,Cys/Trp,22.6,22.30,0.30,5.37,Low,NaN,NaN,NaN,NaN,.,ENST00000568584,-,TGT,3,2,.,0.91255,.,0.0,0.92824,D,1.0,0.90584,D,0.998,0.88582,D,0.000003,0.62929,U,0.000000,0.999882,0.81001,D,simple_aae,C354W,.,.,.,-1.98,0.85247,D,-6.84,0.94249,D,0.751,0.75466,0.3131,0.87764,D,0.8180,0.93873,D,9,0.020603,0.43227,T,0.437,0.74446,0.868,0.96212,Q3ZCM7,C354W,Loss of methylation at K350 (P = 0.0838),.,.,0.76919400692,0.77304,T,0.851523,0.15717,0.19276,0.20597,N,AEFBI,0.422079,0.48823,N,0.0406181179914726,0.43713,2.658815,-0.363070660159128,0.26108,1.439964
1,10_000093271_C_A,10_000047331_C_A,Q3ZCM7_C354,354,C354,Q3ZCM7,2,"{12: 'C', 239: 'C', 303: 'C', 354: 'C'}",C,F,Cys/Phe,22.6,21.80,0.80,5.37,Low,NaN,NaN,NaN,NaN,.,ENST00000568584,-,TGT,2,0,.,0.78490,.,0.013,0.66756,D,1.0,0.90584,D,0.998,0.88582,D,0.000003,0.62929,U,0.000000,0.986821,0.81001,D,simple_aae,C354F,.,.,.,-1.95,0.85003,D,-6.83,0.94223,D,0.782,0.77883,0.4271,0.89500,D,0.7835,0.92649,D,9,0.015962,0.36975,T,0.374,0.69594,0.808,0.92509,Q3ZCM7,C354F,Gain of catalytic residue at C354 (P = 0.1094),.,.,0.727896571159,0.71199,T,0.895918,0.18929,0.57914,0.30502,D,AEFBI,0.537556,0.55531,D,0.0988420731411684,0.46409,2.88164,-0.263815814260368,0.29300,1.640415
2,10_000093271_C_G,10_000047331_C_G,Q3ZCM7_C354,354,C354,Q3ZCM7,2,"{12: 'C', 239: 'C', 303: 'C', 354: 'C'}",C,S,Cys/Ser,22.4,19.67,2.73,5.37,Low,NaN,NaN,NaN,NaN,.,ENST00000568584,-,TGT,2,0,.,0.91255,.,0.026,0.74150,D,0.997,0.70673,D,0.968,0.71741,D,0.000003,0.62929,U,0.000000,0.930492,0.81001,D,simple_aae,C354S,.,.,.,-1.87,0.84415,D,-6.21,0.91827,D,0.739,0.74735,0.4621,0.90007,D,0.7769,0.92419,D,9,0.022348,0.45230,T,0.377,0.69846,0.781,0.90537,Q3ZCM7,C354S,Loss of sheet (P = 0.0817),.,.,0.713165700436,0.69052,T,0.799045,0.12954,0.59480,0.30927,D,AEFBI,0.537556,0.55531,D,0.0966724300357427,0.46307,2.873107,-0.265973651554631,0.29227,1.635795


In [15]:
notdbnsfp.head(3)

,pos_id19,pos_id38,pos_ID,matched_UKBID,matched_aapos,matched_index,Ensembl_transcriptid,Amino_acids,aaref,aaalt,CADD_phred_hg38,CADD_phred_hg19,CADDdiff_38minus19,rs_dbSNP151,genename,cds_strand,refcodon,codonpos,codon_degeneracy,SIFT_score,SIFT_converted_rankscore,SIFT_pred,SIFT4G_score,SIFT4G_converted_rankscore,SIFT4G_pred,Polyphen2_HDIV_score,Polyphen2_HDIV_rankscore,Polyphen2_HDIV_pred,Polyphen2_HVAR_score,Polyphen2_HVAR_rankscore,Polyphen2_HVAR_pred,LRT_score,LRT_converted_rankscore,LRT_pred,LRT_Omega,MutationTaster_score,MutationTaster_converted_rankscore,MutationTaster_pred,MutationTaster_model,MutationTaster_AAE,MutationAssessor_score,MutationAssessor_rankscore,MutationAssessor_pred,FATHMM_score,FATHMM_converted_rankscore,FATHMM_pred,PROVEAN_score,PROVEAN_converted_rankscore,PROVEAN_pred,VEST4_score,VEST4_rankscore,MetaSVM_score,MetaSVM_rankscore,MetaSVM_pred,MetaLR_score,MetaLR_rankscore,MetaLR_pred,Reliability_index,M-CAP_score,M-CAP_rankscore,M-CAP_pred,REVEL_score,REVEL_rankscore,MutPred_score,MutPred_rankscore,MutPred_protID,MutPred_AAchange,MutPred_Top5features,MPC_score,MPC_rankscore,PrimateAI_score,PrimateAI_rankscore,PrimateAI_pred,DANN_score,DANN_rankscore,fathmm-MKL_coding_score,fathmm-MKL_coding_rankscore,fathmm-MKL_coding_pred,fathmm-MKL_coding_group,fathmm-XF_coding_score,fathmm-XF_coding_rankscore,fathmm-XF_coding_pred,Eigen-raw_coding,Eigen-raw_coding_rankscore,Eigen-pred_coding,Eigen-PC-raw_coding,Eigen-PC-raw_coding_rankscore,Eigen-PC-phred_coding
0,10_000093156_C_A,10_000047216_C_A,Q3ZCM7_K392,Q3ZCM7,392,2,ENST00000568584,Lys/Asn,K,N,23.0,22.9,0.1,.,TUBB8,-,AAG,3,2,.,0.91255,.,0.0,0.92824,D,1.0,0.90584,D,0.999,0.9235899999999999,D,0.000010,0.62929,U,0.000000,0.654618,0.81001,D,simple_aae,K392N,.,.,.,-1.98,0.85247,D,-2.6,0.58733,D,0.53,0.6007100000000001,0.3991,0.8908299999999999,D,0.7111,0.90075,D,9,0.009489,0.24855,T,0.491,0.78035,0.718,0.85313,Q3ZCM7,K392N,Loss of MoRF binding (P = 0.0523),.,.,0.751451253891,0.74660,T,0.984728,0.41873,0.38681,0.26043,N,AEFBI,0.295876,0.40587,N,0.0676056746461709,0.44955,2.76036,-0.20748312943205102,0.31266,1.767854
1,10_000093156_C_G,10_000047216_C_G,Q3ZCM7_K392,Q3ZCM7,392,2,ENST00000568584,Lys/Asn,K,N,23.1,22.8,0.3,.,TUBB8,-,AAG,3,2,.,0.91255,.,0.0,0.92824,D,1.0,0.90584,D,0.999,0.9235899999999999,D,0.000010,0.62929,U,0.000000,0.654618,0.81001,D,simple_aae,K392N,.,.,.,-1.98,0.85247,D,-2.6,0.58733,D,0.53,0.6007100000000001,0.3991,0.8908299999999999,D,0.7111,0.90075,D,9,0.008795,0.23196,T,0.491,0.78035,0.718,0.85313,Q3ZCM7,K392N,Loss of MoRF binding (P = 0.0523),.,.,0.751451253891,0.74660,T,0.984663,0.41803,0.40406,0.26429,N,AEFBI,0.302556,0.41077,N,0.0676056746461709,0.44955,2.76036,-0.20748312943205102,0.31266,1.767854
2,10_000093157_T_A,10_000047217_T_A,Q3ZCM7_K392,Q3ZCM7,392,2,ENST00000568584,Lys/Met,K,M,23.2,23.2,0.0,.,TUBB8,-,AAG,2,0,.,0.91255,.,0.0,0.92824,D,1.0,0.90584,D,0.999,0.9235899999999999,D,0.000010,0.62929,U,0.000000,0.820833,0.81001,D,simple_aae,K392M,.,.,.,-2.08,0.86010,D,-3.14,0.66549,D,0.568,0.62442,0.5527,0.9127,D,0.8388,0.9461,D,9,0.01492,0.35345,T,0.505,0.78906,0.698,0.83491,Q3ZCM7,K392M,Loss of ubiquitination at K392 (P = 0.0326),.,.,0.727143764496,0.71088,T,0.964042,0.29699,0.52831,0.29225,D,AEFBI,0.364324,0.45291,N,0.31076238698785996,0.56701,3.8352209999999998,-0.0235518622390073,0.38654,2.280427


# [6b] MERGING CADD37 and 38 DETECTED with dbNSFP detected df

In [16]:
merdet37 = pd.merge(detdbnsfp, det37, how='inner', on=['pos_id19'])

mergeDET = pd.merge(merdet37, det38, how='inner', on=['pos_id38'])

In [17]:
mergeDET.head(3)

,pos_id19,pos_id38,pos_ID,matched_aapos,matched_target,matched_UKBID,matched_index,pos_dict,aaref,aaalt,Amino_acids,CADD_phred_hg38,CADD_phred_hg19,CADDdiff_38minus19,Cys_reactivity,Cys_react_threshold,Cys_target_label,Lys_reactivity,Lys_react_threshold,Lys_target_label,rs_dbSNP151,Ensembl_transcriptid,cds_strand,refcodon,codonpos,codon_degeneracy,SIFT_score,SIFT_converted_rankscore,SIFT_pred,SIFT4G_score,SIFT4G_converted_rankscore,SIFT4G_pred,Polyphen2_HDIV_score,Polyphen2_HDIV_rankscore,Polyphen2_HDIV_pred,Polyphen2_HVAR_score,Polyphen2_HVAR_rankscore,Polyphen2_HVAR_pred,LRT_score,LRT_converted_rankscore,LRT_pred,LRT_Omega,MutationTaster_score,MutationTaster_converted_rankscore,MutationTaster_pred,MutationTaster_model,MutationTaster_AAE,MutationAssessor_score,MutationAssessor_rankscore,MutationAssessor_pred,FATHMM_score,FATHMM_converted_rankscore,FATHMM_pred,PROVEAN_score,PROVEAN_converted_rankscore,PROVEAN_pred,VEST4_score,VEST4_rankscore,MetaSVM_score,MetaSVM_rankscore,MetaSVM_pred,MetaLR_score,MetaLR_rankscore,MetaLR_pred,Reliability_index,M-CAP_score,M-CAP_rankscore,M-CAP_pred,REVEL_score,REVEL_rankscore,MutPred_score,MutPred_rankscore,MutPred_protID,MutPred_AAchange,MutPred_Top5features,MPC_score,MPC_rankscore,PrimateAI_score,PrimateAI_rankscore,PrimateAI_pred,DANN_score,DANN_rankscore,fathmm-MKL_coding_score,fathmm-MKL_coding_rankscore,fathmm-MKL_coding_pred,fathmm-MKL_coding_group,fathmm-XF_coding_score,fathmm-XF_coding_rankscore,fathmm-XF_coding_pred,Eigen-raw_coding,Eigen-raw_coding_rankscore,Eigen-pred_coding,Eigen-PC-raw_coding,Eigen-PC-raw_coding_rankscore,Eigen-PC-phred_coding,GC_hg19,CpG_hg19,priPhCons_hg19,mamPhCons_hg19,verPhCons_hg19,priPhyloP_hg19,mamPhyloP_hg19,verPhyloP_hg19,GerpRS_hg19,GerpRSpval_hg19,GerpN_hg19,GerpS_hg19,Dist2Mutation_hg19,Freq100bp_hg19,Rare100bp_hg19,Sngl100bp_hg19,Freq1000bp_hg19,Rare1000bp_hg19,Sngl1000bp_hg19,Freq10000bp_hg19,Rare10000bp_hg19,Sngl10000bp_hg19,RawScore_hg19,PHRED_hg19,GC_hg38,CpG_hg38,priPhCons_hg38,mamPhCons_hg38,verPhCons_hg38,priPhyloP_hg38,mamPhyloP_hg38,verPhyloP_hg38,GerpRS_hg38,GerpRSpval_hg38,GerpN_hg38,GerpS_hg38,Dist2Mutation_hg38,Freq100bp_hg38,Rare100bp_hg38,Sngl100bp_hg38,Freq1000bp_hg38,Rare1000bp_hg38,Sngl1000bp_hg38,Freq10000bp_hg38,Rare10000bp_hg38,Sngl10000bp_hg38,RawScore_hg38,PHRED_hg38
0,10_000093270_A_C,10_000047330_A_C,Q3ZCM7_C354,354,C354,Q3ZCM7,2,"{12: 'C', 239: 'C', 303: 'C', 354: 'C'}",C,W,Cys/Trp,22.6,22.30,0.30,5.37,Low,NaN,NaN,NaN,NaN,.,ENST00000568584,-,TGT,3,2,.,0.91255,.,0.0,0.92824,D,1.0,0.90584,D,0.998,0.88582,D,0.000003,0.62929,U,0.000000,0.999882,0.81001,D,simple_aae,C354W,.,.,.,-1.98,0.85247,D,-6.84,0.94249,D,0.751,0.75466,0.3131,0.87764,D,0.8180,0.93873,D,9,0.020603,0.43227,T,0.437,0.74446,0.868,0.96212,Q3ZCM7,C354W,Loss of methylation at K350 (P = 0.0838),.,.,0.76919400692,0.77304,T,0.851523,0.15717,0.19276,0.20597,N,AEFBI,0.422079,0.48823,N,0.0406181179914726,0.43713,2.658815,-0.363070660159128,0.26108,1.439964,0.470199,0.053333,0.084,0.084,1.0,0.094,0.095,0.877,NaN,NaN,0.0,0.0,38936.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,24.0,137.0,2.381671,22.30,0.470199,0.053333,0.075,0.075,0.999,-0.918,-0.893,0.492,8890.65,0.0,13.7,13.7,12.0,0.0,0.0,12.0,0.0,0.0,95.0,12.0,31.0,944.0,2.794161,22.6
1,10_000093271_C_A,10_000047331_C_A,Q3ZCM7_C354,354,C354,Q3ZCM7,2,"{12: 'C', 239: 'C', 303: 'C', 354: 'C'}",C,F,Cys/Phe,22.6,21.80,0.80,5.37,Low,NaN,NaN,NaN,NaN,.,ENST00000568584,-,TGT,2,0,.,0.78490,.,0.013,0.66756,D,1.0,0.90584,D,0.998,0.88582,D,0.000003,0.62929,U,0.000000,0.986821,0.81001,D,simple_aae,C354F,.,.,.,-1.95,0.85003,D,-6.83,0.94223,D,0.782,0.77883,0.4271,0.89500,D,0.7835,0.92649,D,9,0.015962,0.36975,T,0.374,0.69594,0.808,0.92509,Q3ZCM7,C354F,Gain of catalytic residue at C354 (P = 0.1094),.,.,0.727896571159,0.71199,T,0.895918,0.18929,0.57914,0.30502,D,AEFBI,0.537556,0.55531,D,0.0988420731411684,0.46409,2.88164,-0.263815814260368,0.29300,1.640415,0.463576,0.053333,0.083,0.082,1.0,0.110,0.109,3.127,NaN,NaN,0.0,0.0,38936.0,NaN,NaN,NaN,NaN,NaN,Na

# [6c] MERGING CADD37 and 38 NOT_DETECTED with dbNSFP not_detected df

In [18]:
mernot37 = pd.merge(notdbnsfp, not37, how='inner', on=['pos_id19'])

mergeNOT = pd.merge(mernot37, not38, how='inner', on=['pos_id38'])

In [19]:
mergeNOT.head(3)

,pos_id19,pos_id38,pos_ID,matched_UKBID,matched_aapos,matched_index,Ensembl_transcriptid,Amino_acids,aaref,aaalt,CADD_phred_hg38,CADD_phred_hg19,CADDdiff_38minus19,rs_dbSNP151,genename,cds_strand,refcodon,codonpos,codon_degeneracy,SIFT_score,SIFT_converted_rankscore,SIFT_pred,SIFT4G_score,SIFT4G_converted_rankscore,SIFT4G_pred,Polyphen2_HDIV_score,Polyphen2_HDIV_rankscore,Polyphen2_HDIV_pred,Polyphen2_HVAR_score,Polyphen2_HVAR_rankscore,Polyphen2_HVAR_pred,LRT_score,LRT_converted_rankscore,LRT_pred,LRT_Omega,MutationTaster_score,MutationTaster_converted_rankscore,MutationTaster_pred,MutationTaster_model,MutationTaster_AAE,MutationAssessor_score,MutationAssessor_rankscore,MutationAssessor_pred,FATHMM_score,FATHMM_converted_rankscore,FATHMM_pred,PROVEAN_score,PROVEAN_converted_rankscore,PROVEAN_pred,VEST4_score,VEST4_rankscore,MetaSVM_score,MetaSVM_rankscore,MetaSVM_pred,MetaLR_score,MetaLR_rankscore,MetaLR_pred,Reliability_index,M-CAP_score,M-CAP_rankscore,M-CAP_pred,REVEL_score,REVEL_rankscore,MutPred_score,MutPred_rankscore,MutPred_protID,MutPred_AAchange,MutPred_Top5features,MPC_score,MPC_rankscore,PrimateAI_score,PrimateAI_rankscore,PrimateAI_pred,DANN_score,DANN_rankscore,fathmm-MKL_coding_score,fathmm-MKL_coding_rankscore,fathmm-MKL_coding_pred,fathmm-MKL_coding_group,fathmm-XF_coding_score,fathmm-XF_coding_rankscore,fathmm-XF_coding_pred,Eigen-raw_coding,Eigen-raw_coding_rankscore,Eigen-pred_coding,Eigen-PC-raw_coding,Eigen-PC-raw_coding_rankscore,Eigen-PC-phred_coding,GC_hg19,CpG_hg19,priPhCons_hg19,mamPhCons_hg19,verPhCons_hg19,priPhyloP_hg19,mamPhyloP_hg19,verPhyloP_hg19,GerpRS_hg19,GerpRSpval_hg19,GerpN_hg19,GerpS_hg19,Dist2Mutation_hg19,Freq100bp_hg19,Rare100bp_hg19,Sngl100bp_hg19,Freq1000bp_hg19,Rare1000bp_hg19,Sngl1000bp_hg19,Freq10000bp_hg19,Rare10000bp_hg19,Sngl10000bp_hg19,RawScore_hg19,PHRED_hg19,GC_hg38,CpG_hg38,priPhCons_hg38,mamPhCons_hg38,verPhCons_hg38,priPhyloP_hg38,mamPhyloP_hg38,verPhyloP_hg38,GerpRS_hg38,GerpRSpval_hg38,GerpN_hg38,GerpS_hg38,Dist2Mutation_hg38,Freq100bp_hg38,Rare100bp_hg38,Sngl100bp_hg38,Freq1000bp_hg38,Rare1000bp_hg38,Sngl1000bp_hg38,Freq10000bp_hg38,Rare10000bp_hg38,Sngl10000bp_hg38,RawScore_hg38,PHRED_hg38
0,10_000093156_C_A,10_000047216_C_A,Q3ZCM7_K392,Q3ZCM7,392,2,ENST00000568584,Lys/Asn,K,N,23.0,22.9,0.1,.,TUBB8,-,AAG,3,2,.,0.91255,.,0.0,0.92824,D,1.0,0.90584,D,0.999,0.9235899999999999,D,0.000010,0.62929,U,0.000000,0.654618,0.81001,D,simple_aae,K392N,.,.,.,-1.98,0.85247,D,-2.6,0.58733,D,0.53,0.6007100000000001,0.3991,0.8908299999999999,D,0.7111,0.90075,D,9,0.009489,0.24855,T,0.491,0.78035,0.718,0.85313,Q3ZCM7,K392N,Loss of MoRF binding (P = 0.0523),.,.,0.751451253891,0.74660,T,0.984728,0.41873,0.38681,0.26043,N,AEFBI,0.295876,0.40587,N,0.0676056746461709,0.44955,2.76036,-0.20748312943205102,0.31266,1.767854,0.536424,0.106667,0.550,0.546,1.0,0.110,0.109,1.700,NaN,NaN,0.0,0.0,38936.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,24.0,137.0,2.636298,22.9,0.536424,0.106667,0.046,0.046,1.0,0.247,0.249,2.730,8890.65,0.0,13.7,13.7,40.0,0.0,0.0,8.0,0.0,0.0,91.0,12.0,31.0,930.0,3.029607,23.0
1,10_000093156_C_G,10_000047216_C_G,Q3ZCM7_K392,Q3ZCM7,392,2,ENST00000568584,Lys/Asn,K,N,23.1,22.8,0.3,.,TUBB8,-,AAG,3,2,.,0.91255,.,0.0,0.92824,D,1.0,0.90584,D,0.999,0.9235899999999999,D,0.000010,0.62929,U,0.000000,0.654618,0.81001,D,simple_aae,K392N,.,.,.,-1.98,0.85247,D,-2.6,0.58733,D,0.53,0.6007100000000001,0.3991,0.8908299999999999,D,0.7111,0.90075,D,9,0.008795,0.23196,T,0.491,0.78035,0.718,0.85313,Q3ZCM7,K392N,Loss of MoRF binding (P = 0.0523),.,.,0.751451253891,0.74660,T,0.984663,0.41803,0.40406,0.26429,N,AEFBI,0.302556,0.41077,N,0.0676056746461709,0.44955,2.76036,-0.20748312943205102,0.31266,1.767854,0.536424,0.106667,0.550,0.546,1.0,0.110,0.109,1.700,NaN,NaN,0.0,0.0,38936.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,24.0,137.0,2.627006,22.8,0.536424,0.106667,0.046,0.046,1.0,0.247,0.249,2.730,8890.65,0.0,13.7,13.7,40.0,0.0,0.0,8.0,0.0,0.0,91.0,12.0,31.0,930.0,3.055876,23.1
2,10_000093157_T_A,10_000047217_T_A,Q3ZCM7_K392,Q3ZC

In [22]:
# saving files
mergeDET.to_csv("MERGE_dbNSFP_CADD_DETECTED_CK_104475.csv", index=False)

mergeNOT.to_csv("MERGE_dbNSFP_CADD_NOT_DETECTED_CK_1222911.csv", index=False)

In [21]:
print(mergeDET.shape)
print(mergeNOT.shape)

(104475, 143)
(1222911, 136)


In [25]:
isnaDET = mergeDET.isna().sum()
isnaDET.to_csv("MERGE_DETECTED_isna_sum.csv")

/Users/mariapalafox/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [24]:
isnaNOT = mergeNOT.isna().sum()
isnaNOT.to_csv("MERGE_NOTDETECTED_isna_sum.csv")

/Users/mariapalafox/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


---
---
---
---
---
---
---
---

---
---
---
---
---
---
---
---

---
---
---
---
---
---
---
---


# (done) CADD file correcting pos_id # format (add leading 0's)
- 9 digits with leading zeros for pos_id19 or pos_id38

```python 
def create_pos_id19(df, chrr, pos, ref, alt):
    # variables are colnames of df
    df.loc[:,'pos_id19'] = df[chrr].astype(str) + '_' + \
    df[pos].astype(str) + '_' + df[ref].astype(str) + \
    '_' + df[alt].astype(str)
    return df

def create_pos_id38(df, chrr, pos, ref, alt):
    # variables are colnames of df
    df.loc[:,'pos_id38'] = df[chrr].astype(str) + '_' + \
    df[pos].astype(str) + '_' + df[ref].astype(str) + \
    '_' + df[alt].astype(str)
    return df

# dbNSFP FILES
dbnsfp = pd.read_csv('SCORE_dbNSFP_selectcols_detected_CK_104475.csv', low_memory=False)
notdbnsfp = pd.read_csv('SCORE_dbNSFP_selectcols_NOT_detected_CK_1222911.csv', low_memory=False)

# CADD FILES
det19 = pd.read_csv('SCORE_CADD_37_detected_CK.csv', low_memory=False, converters={'pos_hg19': '{:0>9}'.format})
det38 = pd.read_csv('SCORE_CADD_38_detected_CK.csv', low_memory=False, converters={'pos_hg38': '{:0>9}'.format})
not19 = pd.read_csv('SCORE_CADD_37_NOT_detected_CK.csv', low_memory=False, converters={'pos_hg19': '{:0>9}'.format})
not38 = pd.read_csv('SCORE_CADD_38_NOT_detected_CK.csv', low_memory=False, converters={'pos_hg38': '{:0>9}'.format})
```

## calling function to create pos id with correct # format:
```python
# cadd detected files
det19 = create_pos_id19(det19, 'chr', 'pos_hg19', 'Ref', 'Alt')
det38 = create_pos_id38(det38, 'chr', 'pos_hg38', 'Ref', 'Alt')

# cadd NOTdetected files
not19 = create_pos_id19(not19, 'chr', 'pos_hg19', 'Ref', 'Alt')
not38 = create_pos_id38(not38, 'chr', 'pos_hg38', 'Ref', 'Alt')

# saving files with correct pos_id
det19.to_csv('SCORE_CADD_37_detected_CK.csv', index=False)
det38.to_csv('SCORE_CADD_38_detected_CK.csv',  index=False)
not19.to_csv('SCORE_CADD_37_NOT_detected_CK.csv', index=False)
not38.to_csv('SCORE_CADD_38_NOT_detected_CK.csv',  index=False)
```

---
---
---


# (initial version, see other M html for more details)
- [3] filter CADDv1.4 files with dbNSFP CK positions for MISSENSE consequence only

dbNSFP SCORE file column names (will map to these files using columns shared with CADD files:

**columns detected file:**
    - Amino_acids
    - pos_ID 'Q3ZCM7_C354'
    - pos_id19 or pos_id38_x

**columns not detected file:**
    - pos_id19 or pos_id38
    - Amino_acids
    - pos_ID_falseCKtarget


```python
# !/usr/bin/env python3
# -*- coding: utf-8 -*-

# Pmap M local ipynb code, Pmap_missense_annotations_QC.py
# markdown M local QC of positions from dbNSFP overlapped with CADD37 or 38 annotations

import os
import sys
import pandas as pd


def create_coordinate_id(df, chrr, pos, ref, alt, assembly):
    if assembly == 37:
        df.loc[:,'pos_id19'] = df[chrr].astype(str) + '_' + \
                df[pos].astype(str) + '_' + df[ref].astype(str) + \
                '_' + df[alt].astype(str)
    if assembly == 38:
        df.loc[:,'pos_id38'] = df[chrr].astype(str) + '_' + \
                df[pos].astype(str) + '_' + df[ref].astype(str) + \
                '_' + df[alt].astype(str)
    return df


def format_missense_triple(df, oaacol, naacol):
    #  A|A turns to Ala/Ala
    amino_dict = dict([('A', 'Ala'),('G', 'Gly'), ('I','Ile'), ('L','Leu'), ('P', 'Pro'), ('V','Val'), ('F','Phe'),('W', 'Trp'), ('Y', 'Tyr'), ('D','Asp'),('E','Glu'), ('R','Arg'),('H','His'), ('K','Lys'), ('S','Ser'), ('T', 'Thr'), ('C', 'Cys'), ('M', 'Met'), ('N', 'Asn'), ('Q','Gln')])
    df[oaacol].replace(amino_dict, inplace=True)
    df[naacol].replace(amino_dict, inplace=True)
    ccopy = df[naacol].copy()
    df['Amino_acids'] = df[oaacol].str.cat(ccopy, sep='/')
    return df


def filter_cadd_overlap(df, assembly):
    # [1] filter for missense only
    miss = df[df['Consequence'] == 'NON_SYNONYMOUS'].copy()
    # [2] new pos_id(assembly) to files
    if assembly == 37:
        miss = create_coordinate_id(miss, 'chr', 'pos_hg19', 'Ref', 'Alt', assembly)
    if assembly == 38:
        miss = create_coordinate_id(miss, 'chr', 'pos_hg38', 'Ref', 'Alt', assembly)
    # [3] new missense type column in 3 letter format with '/' sep {oAA, nAA}
    miss = format_missense_triple(miss, 'oAA', 'nAA')

    return miss

    # [4] concat all files from 37: DECT or NOT ... 38: DECT or NOT


def main():
    os.chdir('/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CADDmapped/RESULT_pos_overlap_dbNSFPcoordinates')

    chrls = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X','Y']

    # GRCh37
    for order in chrls:
        chrID = 'chr{}'.format(order)
        file1 = '{}_CADD_GRCh37_DETECTED_CK.csv'.format(chrID)
        out1 = 'MISSENSE_{}_CADD_GRCh37_DETECTED_CK.csv'.format(chrID)
        file2 = '{}_CADD_GRCh37_NOT_DETECTED_CK.csv'.format(chrID)
        out2 = 'MISSENSE_{}_CADD_GRCh37_NOT_DETECTED_CK.csv'.format(chrID)
        df1 = pd.read_csv(file1, low_memory=False)
        df2 = pd.read_csv(file2, low_memory=False)
        df1out = filter_cadd_overlap(df1, 37)
        df2out = filter_cadd_overlap(df2, 37)
        print("saving detected and not detected GRCh37 ", chrID)
        print()
        df1out.to_csv(out1, index=False)
        df2out.to_csv(out2, index=False)

    # GRCh38
    for order in chrls:
        chrID = 'chr{}'.format(order)
        file1 = '{}_CADD_GRCh38_DETECTED_CK.csv'.format(chrID)
        out1 = 'MISSENSE_{}_CADD_GRCh38_DETECTED_CK.csv'.format(chrID)
        file2 = '{}_CADD_GRCh38_NOT_DETECTED_CK.csv'.format(chrID)
        out2 = 'MISSENSE_{}_CADD_GRCh38_NOT_DETECTED_CK.csv'.format(chrID)
        df1 = pd.read_csv(file1)
        df2 = pd.read_csv(file2)
        df1out = filter_cadd_overlap(df1, 38)
        df2out = filter_cadd_overlap(df2, 38)
        print("saving detected and not detected GRCh38 ", chrID)
        print()
        df1out.to_csv(out1, index=False)
        df2out.to_csv(out2, index=False)

main()

```

# RE running GRCh38 sex chrs after fixing cadd source files on hoffman

```python
# !/usr/bin/env python3
# -*- coding: utf-8 -*-

# Pmap M local ipynb code, Pmap_missense_annotations_QC.py
# markdown M local QC of positions from dbNSFP overlapped with CADD37 or 38 annotations

import os
import sys
import pandas as pd

""" 
dbNSFP SCORE file column names:

columns detected file:
    Amino_acids
    pos_ID 'Q3ZCM7_C354'
    pos_id19 or pos_id38_x

columns not detected file:
    pos_id19 or pos_id38
    Amino_acids,
    pos_ID_falseCKtarget
"""

def create_coordinate_id(df, chrr, pos, ref, alt, assembly):
    if assembly == 37:
        df.loc[:,'pos_id19'] = df[chrr].astype(str) + '_' + \
                df[pos].astype(str) + '_' + df[ref].astype(str) + \
                '_' + df[alt].astype(str)
    if assembly == 38:
        df.loc[:,'pos_id38'] = df[chrr].astype(str) + '_' + \
                df[pos].astype(str) + '_' + df[ref].astype(str) + \
                '_' + df[alt].astype(str)
    return df


def format_missense_triple(df, oaacol, naacol):
    #  A|A turns to Ala/Ala
    amino_dict = dict([('A', 'Ala'),('G', 'Gly'), ('I','Ile'), ('L','Leu'), ('P', 'Pro'), ('V','Val'), ('F','Phe'),('W', 'Trp'), ('Y', 'Tyr'), ('D','Asp'),('E','Glu'), ('R','Arg'),('H','His'), ('K','Lys'), ('S','Ser'), ('T', 'Thr'), ('C', 'Cys'), ('M', 'Met'), ('N', 'Asn'), ('Q','Gln')])
    df[oaacol].replace(amino_dict, inplace=True)
    df[naacol].replace(amino_dict, inplace=True)
    ccopy = df[naacol].copy()
    df['Amino_acids'] = df[oaacol].str.cat(ccopy, sep='/')
    return df


def filter_cadd_overlap(df, assembly):
    # [1] filter for missense only
    miss = df[df['Consequence'] == 'NON_SYNONYMOUS'].copy()
    # [2] new pos_id(assembly) to files
    if assembly == 37:
        miss = create_coordinate_id(miss, 'chr', 'pos_hg19', 'Ref', 'Alt', assembly)
    if assembly == 38:
        miss = create_coordinate_id(miss, 'chr', 'pos_hg38', 'Ref', 'Alt', assembly)
    # [3] new missense type column in 3 letter format with '/' sep {oAA, nAA}
    miss = format_missense_triple(miss, 'oAA', 'nAA')

    return miss

    # [4] concat all files from 37: DECT or NOT ... 38: DECT or NOT


def main():
    os.chdir('/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CADDmapped/RESULT_pos_overlap_dbNSFPcoordinates')
    chrls = ['X', 'Y']
    # GRCh38
    for order in chrls:
        chrID = 'chr{}'.format(order)
        file1 = '{}_CADD_GRCh38_DETECTED_CK.csv'.format(chrID)
        out1 = 'MISSENSE_{}_CADD_GRCh38_DETECTED_CK.csv'.format(chrID)
        file2 = '{}_CADD_GRCh38_NOT_DETECTED_CK.csv'.format(chrID)
        out2 = 'MISSENSE_{}_CADD_GRCh38_NOT_DETECTED_CK.csv'.format(chrID)
        df1 = pd.read_csv(file1)
        df2 = pd.read_csv(file2)
        df1out = filter_cadd_overlap(df1, 38)
        df2out = filter_cadd_overlap(df2, 38)
        print("saving detected and not detected GRCh38 ", chrID)
        print()
        df1out.to_csv(out1, index=False)
        df2out.to_csv(out2, index=False)

main()
```